In [13]:
import pandas as pd
import json

Preperation of the player data:

Source: https://datascienceparichay.com/article/split-pandas-column-of-lists-into-multiple-columns/ [04.11.2021]

Source: https://stackoverflow.com/questions/11346283/renaming-column-names-in-pandas [11.11.2021]

Source: https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html [11.11.2021]

Source: https://stackoverflow.com/questions/44442831/create-multiindex-from-existing-dataframe [18.11.2021]

Source: https://www.kite.com/python/answers/how-to-replace-each-empty-string-in-a-pandas-dataframe-with-nan-in-python [20.11.2021]

Create a dataframe for every player

In [14]:
cols = ['player1','player2','player3','player4','player5','player6','player7',
          'player8','player9','player10']

player1_df = pd.DataFrame([])
player2_df = pd.DataFrame([])
player3_df = pd.DataFrame([])
player4_df = pd.DataFrame([])
player5_df = pd.DataFrame([])
player6_df = pd.DataFrame([])
player7_df = pd.DataFrame([])
player8_df = pd.DataFrame([])
player9_df = pd.DataFrame([])
player10_df = pd.DataFrame([])

players = [player1_df, player2_df, player3_df, player4_df, player5_df, player6_df, player7_df, player8_df, player9_df, player10_df]

- Reading in all game files
- Make the index: tickount + gamenumber*1000000
- split the player array of the JSON

In [20]:
def preparePlayerDf():
  path = 'drive/MyDrive/BA_JSON/'
  files = ['cs02_player.json', 'cs03_player.json', 'cs04_player.json', 
           'cs06_player.json', 'cs07_player.json', 'cs08_player.json', 
           'cs09_player.json', 'cs10_player.json', 'cs12_player.json']
           #'cs01_player.json', 'cs11_player.json', 'cs13_player.json'
           #these three are exluded due to wrong pov input

  game_index_factor = 1000000
  game_index = 0

  for y in files:
    df = pd.read_json(path + y)
    df['tickCount'] += game_index * game_index_factor
    game_index += 1
    tick_df = df
    tick_df = tick_df.set_index('tickCount')
    split_df = pd.DataFrame(df['player'].to_list(), columns=cols)
    df = pd.concat([df, split_df], axis=1)
    df = df.drop('player', axis=1)
    df = prepareGamePlayers(df, tick_df)

Split the players in player data frames by name due to inconsitencies with the id and fill set the values for dead/leaved players

In [16]:
def prepareGamePlayers(df, tick_df):
  player = prepareSinglePlayer(df, cols[0])
  for x in cols[1:]:
    single_player = prepareSinglePlayer(df, x)
    player = pd.concat([player, single_player], axis=0)
  
  names = player['name'].unique()

  for i in range(10):
    p = player[player['name'] == names[i]]
    p = p.set_index('tickCount')
    p = pd.concat([tick_df, p], axis=1)
    p = p.drop(['player'], axis=1)
    p = p.fillna(0)
    p = p.drop(['userID', 'name'], axis=1)

    if players[i].empty:
      players[i] = p
    else:
      players[i] = pd.concat([players[i], p], axis=0)

Handling the NaNs of a single player df

In [17]:
def prepareSinglePlayer(df, player):
  p_df = df[['tickCount', player]]
  p_df = p_df.dropna()
  p_df = p_df.reset_index(drop=True)
  split_df = pd.DataFrame(p_df[player].to_list())
  p_df = pd.concat([p_df, split_df], axis=1)
  p_df = p_df.drop([player, 'eyeAngeles'], axis=1)
  p_df = p_df.replace(r'^\s*$', 0, regex=True)
  p_df = p_df.fillna(0)
  return p_df

In [21]:
preparePlayerDf()

Looking at the data

In [22]:
players[9]

,teamNumber,armor,health,placeName,hasC4,currentEquipmentValue,flashDuration,hasDefuser,isScoped,isSpotted,weapon,hasSpotted,POVPlayer
tickCount,,,,,,,,,,,,,
718,0.0,0.0,0.0,0,0,0.0,0.0,0,0,0,0,0,0
719,0.0,0.0,0.0,0,0,0.0,0.0,0,0,0,0,0,0
720,0.0,0.0,0.0,0,0,0.0,0.0,0,0,0,0,0,0
721,0.0,0.0,0.0,0,0,0.0,0.0,0,0,0,0,0,0
722,0.0,0.0,0.0,0,0,0.0,0.0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8265186,3.0,98.0,58.0,TunnelStairs,False,5900.0,0.0,True,False,False,weapon_m4a1_silencer,False,False
8265188,3.0,98.0,58.0,TunnelStairs,False,5900.0,0.0,True,False,False,weapon_m4a1_silencer,False,False
8265190,3.0,98.0,58.0,TunnelStairs,False,5900.0,0.0,True,False,False,weapon_m4a1_silencer,False,False


Correcting the datatypes of the columns

In [23]:
for player in players:
  player['hasC4'] = [0 if x == False else 1 for x in player['hasC4']]
  player['hasDefuser'] = [0 if x == False else 1 for x in player['hasDefuser']]
  player['isScoped'] = [0 if x == False else 1 for x in player['isScoped']]
  player['isSpotted'] = [0 if x == False else 1 for x in player['isSpotted']]
  player['hasSpotted'] = [0 if x == False else 1 for x in player['hasSpotted']]
  player['POVPlayer'] = [0 if x == False else 1 for x in player['POVPlayer']]
  player['flashDuration'] = [0 if x == 0 else 1 for x in player['flashDuration']]
  player['placeName'] = ['0' if x == 0 else x for x in player['placeName']]
  player['weapon'] = ['0' if x == 0 else x for x in player['weapon']]

In [24]:
players[9]

,teamNumber,armor,health,placeName,hasC4,currentEquipmentValue,flashDuration,hasDefuser,isScoped,isSpotted,weapon,hasSpotted,POVPlayer
tickCount,,,,,,,,,,,,,
718,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0
719,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0
720,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0
721,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0
722,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8265186,3.0,98.0,58.0,TunnelStairs,0,5900.0,0,1,0,0,weapon_m4a1_silencer,0,0
8265188,3.0,98.0,58.0,TunnelStairs,0,5900.0,0,1,0,0,weapon_m4a1_silencer,0,0
8265190,3.0,98.0,58.0,TunnelStairs,0,5900.0,0,1,0,0,weapon_m4a1_silencer,0,0


Save the dataframes as csv

In [25]:
players[0].to_csv('drive/MyDrive/Bachelorarbeit/Datensatz/Spielerdaten/player1_data.csv')
players[1].to_csv('drive/MyDrive/Bachelorarbeit/Datensatz/Spielerdaten/player2_data.csv')
players[2].to_csv('drive/MyDrive/Bachelorarbeit/Datensatz/Spielerdaten/player3_data.csv')
players[3].to_csv('drive/MyDrive/Bachelorarbeit/Datensatz/Spielerdaten/player4_data.csv')
players[4].to_csv('drive/MyDrive/Bachelorarbeit/Datensatz/Spielerdaten/player5_data.csv')
players[5].to_csv('drive/MyDrive/Bachelorarbeit/Datensatz/Spielerdaten/player6_data.csv')
players[6].to_csv('drive/MyDrive/Bachelorarbeit/Datensatz/Spielerdaten/player7_data.csv')
players[7].to_csv('drive/MyDrive/Bachelorarbeit/Datensatz/Spielerdaten/player8_data.csv')
players[8].to_csv('drive/MyDrive/Bachelorarbeit/Datensatz/Spielerdaten/player9_data.csv')
players[9].to_csv('drive/MyDrive/Bachelorarbeit/Datensatz/Spielerdaten/player10_data.csv')

Test if everything works correct

In [26]:
test = pd.read_csv('drive/MyDrive/Bachelorarbeit/Datensatz/Spielerdaten/player10_data.csv')
test

,tickCount,teamNumber,armor,health,placeName,hasC4,currentEquipmentValue,flashDuration,hasDefuser,isScoped,isSpotted,weapon,hasSpotted,POVPlayer
0,718,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0
1,719,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0
2,720,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0
3,721,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0
4,722,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
751096,8265186,3.0,98.0,58.0,TunnelStairs,0,5900.0,0,1,0,0,weapon_m4a1_silencer,0,0
751097,8265188,3.0,98.0,58.0,TunnelStairs,0,5900.0,0,1,0,0,weapon_m4a1_silencer,0,0
751098,8265190,3.0,98.0,58.0,TunnelStairs,0,5900.0,0,1,0,0,weapon_m4a1_silencer,0,0
751099,8265192,3.0,98.0,58.0,TunnelStairs,0,5900.0,0,1,0,0,weapon_m4a1_silencer,0,0
